---
# **LAB 4 - CUDA memories**
---

# ▶️ CUDA tools...

In [ ]:
!nvidia-smi

Tue Jan 27 15:00:29 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import numpy as np
import numba
from numba import cuda
import warnings
warnings.filterwarnings("ignore")

print(np.__version__)
print(numba.__version__)

cuda.detect()



2.0.2
0.60.0
Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 4
                              PCI Bus ID: 0
                                    UUID: GPU-08256c08-312d-14b3-b5c0-5a02cbb6708e
                                Watchdog: Disabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


True

In [3]:
# Suppress Numba deprecation and performance warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPerformanceWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

Utils for compiling and running Numba CUDA code.

In [4]:
from numba import cuda

def mem_snapshot(print=True):
    free, total = cuda.current_context().get_memory_info()
    used = total - free
    if print:
        # print GPU memory info
        print("\nMemory occupancy:")
        print(f"    GPU total: {total/1024**3:.3f} GB")
        print(f"    GPU free : {free/1024**3:.3f} GB")
        print(f"    GPU used : {used/1024**3:.3f} GB")
    else:
        return used, free, total

# Quick device spec report (Numba)
def device_info(show=True):
    dev = cuda.get_current_device()   # raises if no CUDA device
    _, total = cuda.current_context().get_memory_info() 
    
    if show:
        print("Device object repr:", dev)
        print("Device name:          ", getattr(dev, "name", "<unknown>"))
        print("Compute capability:   ", getattr(dev, "compute_capability", "<unknown>"))

    # Common numeric properties (use getattr to avoid attribute errors)
    props = {
        "  multi_processor_(SM)_count": ["MULTIPROCESSOR_COUNT"],
        "  max_threads_per_block": ["MAX_THREADS_PER_BLOCK"],
        "  max_block_dim_x":       ["MAX_BLOCK_DIM_X"],
        "  max_block_dim_y":       ["MAX_BLOCK_DIM_Y"],
        "  max_block_dim_z":       ["MAX_BLOCK_DIM_Z"],
        "  max_grid_dim_x":        ["MAX_GRID_DIM_X"],
        "  max_grid_dim_y":        ["MAX_GRID_DIM_Y"],
        "  max_grid_dim_z":        ["MAX_GRID_DIM_Z"],
        "  max_shared_memory_per_block (bytes)": ["MAX_SHARED_MEMORY_PER_BLOCK"],
        "  max_shared_memory_per_SM (bytes)": ["MAX_SHARED_MEMORY_PER_MULTIPROCESSOR"],
        "  warp_size":             ["WARP_SIZE"],
        "  compute_capability":    ["COMPUTE_CAPABILITY", "cc"],
    }
    
    feats = {}
    label = "  total_memory (bytes)"
    feats[label] = total
    if show:
        print(f"{label:40}: {total}")
    for label, keys in props.items():
        val = None
        for k in keys:
            val = getattr(dev, k, None)
            feats[k] = val
            if val is not None:
                break
        if show:
            print(f"{label:40}: {val}")
    return feats

_ = device_info()

Device object repr: <CUDA device 0 'b'Tesla T4''>
Device name:           b'Tesla T4'
Compute capability:    (7, 5)
  total_memory (bytes)                  : 15828320256
  multi_processor_(SM)_count            : 40
  max_threads_per_block                 : 1024
  max_block_dim_x                       : 1024
  max_block_dim_y                       : 1024
  max_block_dim_z                       : 64
  max_grid_dim_x                        : 2147483647
  max_grid_dim_y                        : 65535
  max_grid_dim_z                        : 65535
  max_shared_memory_per_block (bytes)   : 49152
  max_shared_memory_per_SM (bytes)      : 65536
  warp_size                             : 32
  compute_capability                    : None


# ✅ Parallel reduction with shared memory


## ↘️ TODO...

### Block Reduction with Shared Memory (Numba CUDA)

-   Implement a **block-level reduction** kernel in **Numba CUDA** using **shared memory (SMEM)**.

    -   Input: 1D array `array`
    -   Output: 1D array `out` with **one partial sum per block**
    -   Each block loads its elements into shared memory, then reduces them to a single sum.

<br> 🔹 **Learning Objectives**

-   Allocate and use **shared memory** in a CUDA kernel
-   Use `cuda.grid(1)` to compute a **global index**
-   Apply the **standard reduction loop**: $$
    \text{stride} = \frac{\text{blockDim.x}}{2}, \frac{\text{blockDim.x}}{4}, \dots, 1
    $$
-   **Synchronize threads** with `cuda.syncthreads()`
-   **Write** **one result** per block into `out`

<br> 🔹 **Thread tasks...**

-   Each thread:

    -   **Loads** one element into shared memory

    -   **Performs reduction** in shared memory

    -   Thread `tid == 0` **writes the result** for the block

<br> 🔹 **Allocate Shared Memory**

-   Shared array must have **compile-time constant size**:

```{python}
SMEM_SIZE = 1024
smem = cuda.shared.array(SMEM_SIZE, dtype=np.float32)
```

-   This creates one shared buffer per block

- Nest steps:
    - Load Data into Shared Memory
    - Shared Memory Reduction Loop
    - Write One Result per Block

- Template...

```{python}
import numpy as np
from numba import cuda

TPB = 256
SMEM_SIZE = TPB

@cuda.jit
def blockParReduceSMEM(array, out, n):
    tid = cuda.threadIdx.x
    i = cuda.grid(1)

    smem = cuda.shared.array(SMEM_SIZE, dtype=np.float32)

    # TODO: load (with bounds check + padding)
    # TODO: cuda.syncthreads()

    # TODO: reduction loop

    # TODO: write out[blockIdx.x]
```

In [6]:
import numpy as np
from numba import cuda
import time
# Parellel reduction with no divergence
@cuda.jit
def blockParReduceSMEM(in_arr, out_arr):
    tid = cuda.threadIdx.x
    idx = cuda.grid(1) # global index
    n = len(in_arr)
    stride = 1
    base = cuda.blockIdx.x * cuda.blockDim.x

    if idx >= n:
            return

    while stride < cuda.blockDim.x:
        index = 2 * stride * tid
        if index < cuda.blockDim.x:
            in_arr[base + index] += in_arr[base + index + stride]
        stride *= 2
        cuda.syncthreads()

    if tid == 0:
        out_arr[cuda.blockIdx.x] = in_arr[base]

# ----------------------------
# host-side usage
# ----------------------------
blockSize = 1024;               # block dim 1D
numBlock = 1024*1024          # grid dim 1D
n = blockSize * numBlock;       # array dim

# prepare data
a = np.ones(n, dtype=np.int32)
a_d = cuda.to_device(a)
b_d = cuda.device_array(numBlock, dtype=np.int32)

# numpy sum time
tic = time.time()
s_cpu = a.sum()
toc = time.time()
print(f"Numpy sum time: {toc - tic:.4f} seconds")

# launch kernel
t0 = time.perf_counter()
blockParReduceSMEM[numBlock, blockSize](a_d, b_d)
cuda.synchronize()
t1 = time.perf_counter()
print(f"Kernel execution time: {t1 - t0:.4f} seconds")
print("speedup over numpy:", (toc - tic) / (t1 - t0))

# copy result back to host
b = b_d.copy_to_host()
s_gpu = b.sum()
print('GPU sum = ', s_gpu, ' CPU sum = ', s_cpu)
assert s_cpu == s_gpu, "Error! Reduction result does not match!"


ERROR:numba.cuda.cudadrv.driver:Call to cuMemAlloc results in CUDA_ERROR_ILLEGAL_ADDRESS


CudaAPIError: [700] Call to cuMemAlloc results in CUDA_ERROR_ILLEGAL_ADDRESS

# ✅ Matrix multiplication with shared memory (smem)


In [ ]:
import numpy as np
from numba import cuda,  float32
import time

@cuda.jit
def matMul(A, B, C):
    """Perform square matrix multiplication of C = A * B.

    Parameters
    ----------
    A : 2D array
        Input matrix A
    B : 2D array
        Input matrix B
    C : 2D array
        Output matrix C
    """

    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.0
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

#  Matrix sizes:
#     A: (M x P) float32
#     B: (P x N) float32
#     C: (M x N) float32

TPB = 16  # Threads per block
N = TPB * 1000  # Number of rows
M = TPB * 1000  # Number of columns
P = TPB * 1000  # Inner dimension

# Initialize matrices
A = np.ones((N,P), dtype=np.float32)   # A matrix 
B = 2* np.ones((P,M), dtype=np.float32)   # B matrix 
C = np.zeros((N, M), dtype=np.float32)  # Output matrix

# verify numpy sum time
tic = time.time()
C_cpu = A @ B
toc = time.time()
print(f"Numpy sum time: {toc - tic:.4f} seconds")

# GPU setup
threads = (TPB, TPB)
blocks = ((N + (threads[0] - 1)) // threads[0], (M + (threads[1] - 1)) // threads[1])
d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.device_array((N, M), dtype=A.dtype)

# launch kernels and time
t0 = time.perf_counter()
matMul[blocks, threads](d_A, d_B, d_C)
cuda.synchronize()
t1 = time.perf_counter()
print(f"Kernel blockParReduceSMEM execution time: {t1 - t0:.4f} seconds")
print("speedup over numpy:", (toc - tic) / (t1 - t0))

# Final reduction on CPU
C = d_C.copy_to_host()  # Final reduction in CPU

# Verify correctness
print(C)
print(C_cpu)


## ↘️ TODO...

<br> 🔹 **Problem Definition**

-   Given: $A$ of shape $(N, P)$, $B$ of shape $(P, M)$

-   Compute: 
$$
      C = A \cdot B \quad \text{of shape } (N, M) 
$$

-   Elementwise: 
    $$
      C[y, x] = \sum_{k=0}^{P-1} A[y, k] \cdot B[k, x]
    $$

-   using **TPB×TPB tiles** loaded into shared memory

<br> 🔹 **Learning Objectives**

-   Launch a kernel with a **2D grid** and **2D blocks**
-   Use `cuda.shared.array()` to create **shared-memory tiles**
-   Implement a tiled dot product using **sweep over tiles**
-   Use `cuda.syncthreads()` correctly
-   Validate GPU results against NumPy (`A @ B`)
-   Measure runtime and compute speedup

<br> 🔹 **CUDA Mapping**

- Each thread computes one element of C:
    - Thread global coordinates:
    ```python
    x, y = cuda.grid(2)
    ```
- Thread local coordinates in the block:

```python
tx = cuda.threadIdx.x
ty = cuda.threadIdx.y
```

- So thread `(tx, ty)` in block `(bx, by)` computes `C[y, x]`

<br> 🔹 **Tiling Strategy (High Level)**

- Instead of reading the full row/column from global memory, we:
	1.	Load a tile of A into shared memory sA
	2.	Load a tile of B into shared memory sB
	3.	Multiply-accumulate inside the tile
	4.	Repeat for all tiles along the inner dimension

- This reduces global memory traffic

<br> 🔹 **Exercise Tasks**

- Your tasks:
	1.	Create shared-memory tiles sA and sB
	2.	Compute global coordinates (x, y) using cuda.grid(2)
	3.	Loop over tiles along the inner dimension
	4.	Load tiles from A and B into shared memory
	5.	Synchronize threads
	6.	Compute partial dot product using shared memory
	7.	Synchronize again before loading next tile
	8.	Store final result in C[y, x]



```{python}
import numpy as np
from numba import cuda, float32

TPB = 16

@cuda.jit
def matMul_SMEM(A, B, C):
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    # TODO: compute how many tiles along the inner dimension
    # tiles = ...

    tmp = float32(0.0)

    # TODO: loop over tiles
    # for i in range(tiles):

        # TODO: load sA[ty, tx] from A (with bounds)
        # TODO: load sB[ty, tx] from B (with bounds)

        # TODO: synchronize
        # cuda.syncthreads()

        # TODO: compute partial dot product over j
        # for j in range(TPB):
        #     tmp += sA[ty, j] * sB[j, tx]

        # TODO: synchronize before next tile
        # cuda.syncthreads()

    # TODO: store result in C (with bounds)
```



In [ ]:
import numpy as np
from numba import cuda, float32

TPB = 16

@cuda.jit
def matMul_SMEM(A, B, C):
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    # TODO: compute how many tiles along the inner dimension
    # tiles = ...

    tmp = float32(0.0)

    # TODO: loop over tiles
    # for i in range(tiles):

        # TODO: load sA[ty, tx] from A (with bounds)
        # TODO: load sB[ty, tx] from B (with bounds)

        # TODO: synchronize
        # cuda.syncthreads()

        # TODO: compute partial dot product over j
        # for j in range(TPB):
        #     tmp += sA[ty, j] * sB[j, tx]

        # TODO: synchronize before next tile
        # cuda.syncthreads()

    # TODO: store result in C (with bounds)

# ✅ Convolution with smem

## ↘️ TODO...


- Host code that:

  - allocates input `data`, `mask`, and output arrays
  - computes a reference result with `np.convolve(..., mode="same")`
  - runs the shared-memory kernel
  - runs the basic kernel
  - prints timings and speed-ups
  - checks maximum absolute error between host and device results

You **do not need** to type all code from scratch – focus on **reading & modifying**.

<br> 🔹  **Understand the Parameters**

- Inspect the parameter definitions:

```python
BLOCK_SIZE   = 1024
MASK_RADIUS  = 100
MASK_SIZE    = 2 * MASK_RADIUS + 1
TILE_SIZE    = BLOCK_SIZE + MASK_SIZE - 1  # shared tile length per block

n = 1024 * 1024 * 1024
```


<br> 🔹  **CPU Reference Convolution**

- The host code computes:

```python
h_ref = np.convolve(data, mask, mode='same')
```

<br> 🔹  **Basic Kernel `conv1d_basic`**

Skeleton:

```python
@cuda.jit
def conv1d_basic(result, data, mask):
    i = cuda.grid(1)
    if i >= len(data):
        return

    mask_size = mask.shape[0]
    radius = mask_size // 2

    offset = i - radius
    start = 0 if offset >= 0 else -offset
    end = mask_size if (offset + mask_size) <= n else (n - offset)

    acc = 0.0
    for j in range(start, end):
        acc += data[offset + j] * mask[j]

    result[i] = acc
```

<br> 🔹  **Shared-Memory Kernel `conv1d_shared`**

- Each block loads a **tile** into shared memory:
  - left halo (MASK_RADIUS elements)
  - center (block size elements)
  - right halo (MASK_RADIUS elements)
- Threads then read from shared memory instead of global memory

### Tasks

1. Identify where **left halo**, **center**, and **right halo** are loaded in `conv1d_shared`.
2. Explain why the code calls `cuda.syncthreads()` before performing the the convolution.
3. Compare memory access patterns:
   - basic kernel: global memory
   - shared kernel: global → shared → reused

<br> 🔹  **Launch Configuration**

- The device launch configuration in the host code is:

```python
threads = BLOCK_SIZE
blocks = (n + BLOCK_SIZE - 1) // BLOCK_SIZE
conv1d_shared[blocks, threads](d_result, d_data, d_mask)
```


<br> 🔹 **Timing & Speedup**

At the end, the script prints:

- `t_host` (CPU time)
- `t_dev_shared` (GPU shared-memory mode)
- `t_dev_basic` (GPU basic kernel)
- Speedups: `host / shared`, `basic / shared`
- Maximum absolute errors vs reference

<br> 🔹  **Experiment: Different Masks**

Currently, `mask` is:

```python
mask = np.ones(MASK_SIZE, dtype=np.float32)
```
